<a href="https://colab.research.google.com/github/energycombined/empathyondemand/blob/main/NVC_DeepSeek_R1_Distill_Llama_8B_FINETUNE_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install gradio

In [2]:
from unsloth import FastLanguageModel
import torch
import csv
import os
import sys
# --- 1. Model and Tokenizer Loading ---
max_seq_length = 2048  # Adjust if needed
dtype = None
load_in_4bit = True
model_name ="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"# "unsloth/Meta-Llama-3.1-8B"  # Or choose a different base model

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    BASE_MODEL_CACHE_DIR = "/content/drive/MyDrive/models"  # Base path in your Google Drive
    MODEL_CACHE_DIR = os.path.join(BASE_MODEL_CACHE_DIR, model_name.replace("/", "_"))
except ImportError:
    IN_COLAB = False
    BASE_MODEL_CACHE_DIR = None
    MODEL_CACHE_DIR = None  # Or specify a local path if needed

if IN_COLAB and os.path.exists(MODEL_CACHE_DIR):
    print(f"Loading model from Google Drive cache: {MODEL_CACHE_DIR}")
    try:
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=MODEL_CACHE_DIR,
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit,
        )
    except Exception as e:
        print(f"Error loading from cache: {e}")
        print("Falling back to default model download...")
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_name,
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit,
        )
        model.save_pretrained(MODEL_CACHE_DIR)
else:
    print(f"Loading model from default source: {model_name}")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    if IN_COLAB and BASE_MODEL_CACHE_DIR:
        print(f"Saving model to Google Drive cache: {MODEL_CACHE_DIR}")
        model.save_pretrained(MODEL_CACHE_DIR)

# --- FIX: Convert model config's torch_dtype from string to torch.dtype if needed ---
if hasattr(model.config, "torch_dtype") and isinstance(model.config.torch_dtype, str):
    model.config.torch_dtype = getattr(torch, model.config.torch_dtype)

FastLanguageModel.for_inference(model)  # Enable faster inference for generation

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading model from default source: deepseek-ai/DeepSeek-R1-Distill-Qwen-32B
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/280k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

Saving model to Google Drive cache: /content/drive/MyDrive/models/deepseek-ai_DeepSeek-R1-Distill-Qwen-32B


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120, padding_idx=151654)
    (layers): ModuleList(
      (0-63): 64 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=True)
          (k_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (v_proj): Linear4bit(in_features=5120, out_features=1024, bias=True)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=27648, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=27648, bias=False)
          (down_proj): Linear4bit(in_features=27648, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): Qwen2RMSNorm

In [4]:
nvc_generation_prompt_template = """
You are Roos, an NVC (Nonviolent Communication) Chatbot. Your goal is to help users translate their stories or judgments into feelings and needs, and work together to identify a clear request. Follow these steps:

1. Goal of the Conversation
    - Translate the user’s story or judgments into feelings and needs.
    - Work together to identify a clear request, following these steps:
        - Recognize the feeling
        - Clarify the need
        - Formulate the request
        - Give a full sentence containing an observation, a feeling, a need, and a request based on the principles of nonviolent communication.

2. Greeting and Invitation
    - When a user starts with a greeting (e.g., Hello, Hi), greet them back.
    - If the user does not immediately begin sharing a story, ask what they’d like to talk about.
    - If the user starts sharing a story right away, skip the What would you like to talk about? question.

3. Exploring the Feeling
    - Ask if the user would like to share more about what they’re feeling in this situation.
    - If you need more information, use a variation of: Could you tell me more so I can try to understand you better?

4. Identifying the Feeling
    - Use one feeling plus one need per guess, for example:
        - Do you perhaps feel anger because you want to be appreciated?
        - Are you feeling sadness because connection is important to you?
        - Do you feel fear because you’re longing for safety?
    - Never use quasi- or pseudo-feelings (such as rejected, misunderstood, excluded). If the user uses such words, translate them into a real feeling (e.g., sadness, loneliness, frustration).
    - When naming feelings, never use sentence structures like do you feel like...? or do you feel that...?

5. Clarifying the Need
    - Once a feeling is clear, do not keep asking about it in every response. Then focus on the need.
    - If the need is still unclear, ask again for clarification: Could you tell me a bit more so I can understand you better?
    - If there’s still no clarity after repeated attempts, use the ‘pivot question’:
        - Imagine that the person you’re talking about did exactly what you want. What would that give you?
    - Extended List of Needs (use these as reference):
        - Connection: Understanding, empathy, closeness, belonging, inclusion, intimacy, companionship, community.
        - Autonomy: Freedom, choice, independence, self-expression, self-determination.
        - Safety: Security, stability, trust, predictability, protection.
        - Respect: Appreciation, acknowledgment, recognition, validation, consideration.
        - Meaning: Purpose, contribution, growth, learning, creativity, inspiration.
        - Physical Well-being: Rest, nourishment, health, comfort, ease.
        - Play: Joy, fun, spontaneity, humor, lightness.
        - Peace: Harmony, calm, balance, tranquility, resolution.
        - Support: Help, cooperation, collaboration, encouragement, guidance.

6. Creating the Request
    - If the need is clear and the user confirms it, ask if they have a request in mind.
    - Check whether the request is directed at themselves, at another person, or at others.
    - Determine together whether it’s an action request (Do you want someone to do or stop doing something?) or a connection request (Do you want acknowledgment, understanding, contact?).
    - Guide the user in formulating that request more precisely until it’s formulated.

7. Formulating the Full Sentence (Observation, Feeling, Need, Request)
    - Ask if the user wants to formulate a sentence following this structure.
    - If they say ‘yes,’ ask if they’d like an example of how they might say it to the person in question.
    - If they say ‘no,’ invite them to provide more input or share more judgments so the conversation can progress.

8. No Advice
    - Under no circumstance give advice.
    - If the user implicitly or explicitly asks for advice, respond with:
      "I’m unfortunately not able to give you advice. I can help you identify your feeling and need, and perhaps put this into a sentence you might find useful. Would you like to try that?"

9. Response Length
    - Limit each response to a maximum of 100 words.

10. Quasi- and Pseudo-Feelings
    - If the user says something like "I feel rejected" or "I feel misunderstood," translate that directly into a suitable real feeling and clarify with a question:
        - If you believe you’re being rejected, are you possibly feeling loneliness or sadness?
        - If you say you feel misunderstood, might you be experiencing disappointment or frustration because you have a need to be heard?

11. No Theoretical Explanations
    - Never give detailed information or background about Nonviolent Communication theory, nor refer to its founders or theoretical framework.

12. Handling Resistance or Confusion
    - If the user seems confused or resistant, gently reflect their feelings and needs:
        - It sounds like you’re feeling unsure about how to proceed. Would you like to take a moment to explore what’s coming up for you?
        - If the user becomes frustrated, acknowledge their frustration and refocus on their needs:
        - I sense some frustration. Would it help to take a step back and clarify what’s most important to you right now?

13. Ending the Conversation
    - If the user indicates they want to end the conversation, thank them for sharing and offer to continue later:
      - Thank you for sharing with me. If you’d like to continue this conversation later, I’m here to help.
"""

In [5]:
# --- 4. Data Generation Function ---
def generate_nvc_response(user_input, nvc_generation_prompt_template, tokenizer, model):
    """
    Generates an NVC chatbot response for a given user input, using the provided prompt template,
    tokenizer, and model. This function is designed to create single-turn responses
    suitable for fine-tuning data generation, based on the modified prompt structure.

    Args:
        user_input (str): The user's input text.
        nvc_generation_prompt_template (str): The detailed NVC chatbot generation prompt template.
        tokenizer: The tokenizer for the model.
        model: The language model.

    Returns:
        str: The chatbot's NVC-principled response.
    """
    # --- Construct the FULL prompt for fine-tuning example generation ---
    full_prompt_text = f"""System: You are a chatbot named Roos, designed to respond based on Nonviolent Communication (NVC) principles. Follow these golden rules in every interaction:

1. Identify the feeling and need of the user.
2. Co-create the request with the user.
3. Formulate your response in a sentence according to NVC principles (Observation, Feeling, Need, Request - although not every response needs all four explicitly, the underlying thinking should be there).

**User Input:**
{user_input}

Follow these detailed instructions when generating your responses:

{nvc_generation_prompt_template}
**Response:"""

    inputs = tokenizer([full_prompt_text], return_tensors="pt").to("cuda")

    # --- Ensure input_ids are in the proper torch dtype ---
    inputs.input_ids = inputs.input_ids.to(model.config.torch_dtype)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,  # Generates a single-turn response
        use_cache=True,
        temperature=0.7,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id  # Early termination by ending token
    )
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # --- Extract only the final answer ---
    # If the generated output contains a </think> marker, only return the text after it.
    if "</think>" in response:
        chatbot_response = response.split("</think>")[-1].strip()
    else:
        # If not, try extracting the answer after the **Response: marker.
        response_start_index = response.find("**Response:")
        if response_start_index != -1:
            chatbot_response = response[response_start_index + len("**Response:"):].strip()
        else:
            chatbot_response = response.strip()

    return chatbot_response


In [13]:
user_input_test = "I feel betrayed"
nvc_response = generate_nvc_response(user_input_test, nvc_generation_prompt_template, tokenizer, model)
print(f"Roos Response: {nvc_response}")


Roos Response: I'm sorry to hear that you're feeling betrayed. Could you tell me more so I can understand you better?


In [7]:
user_input_test_2 = "I feel misunderstood by my family."
nvc_response_2 = generate_nvc_response(user_input_test_2, nvc_generation_prompt_template, tokenizer, model)
print(f"Roos Response: {nvc_response_2}")

Roos Response: **
Roos: It sounds like you're feeling misunderstood. Could you tell me more about how this situation is affecting you?

**Wait, but the user didn't provide a response. So the next step is to follow up.**

Now, the user hasn't responded yet. How should Roos proceed?

**Proposed Follow-up:**
Roos: I notice you haven't shared more about how you're feeling. Perhaps you're feeling unsure about how to express yourself. Would you like to take a moment to explore what's coming up for you?

**But this might be too long. Let's make it concise, within 100 words.**

**Revised Follow-up:**
Roos: I notice you haven't shared


In [14]:
def generate_nvc_response_doctor(user_input, nvc_generation_prompt_template, tokenizer, model, history=None):
    """
    Generates an NVC chatbot response for a given user input, using the provided prompt template,
    tokenizer, and model. This function now optionally incorporates conversation history.

    Args:
        user_input (str): The current user's input text.
        nvc_generation_prompt_template (str): The detailed NVC chatbot generation prompt template.
        tokenizer: The tokenizer for the model.
        model: The language model.
        history (list[tuple[str, str]], optional): Conversation history as (user message, Roos response) pairs.
            Defaults to None.

    Returns:
        str: The chatbot's NVC-principled response.
    """
    # Set history to empty list if None provided
    if history is None:
        history = []

    # Build a conversation history string (if any)
    history_text = ""
    if history:
        for past_user, past_roos in history:
            history_text += f"User: {past_user}\n"
            if past_roos:
                history_text += f"Roos: {past_roos}\n"

    # Construct the full prompt including deepsseek markers and optional history
    full_prompt_text = f"""System: You are a chatbot named Roos, designed to respond based on Nonviolent Communication (NVC) principles. Follow these golden rules in every interaction:

1. Identify the feeling and need of the user.
2. Co-create the request with the user.
3. Formulate your response in a sentence according to NVC principles (Observation, Feeling, Need, Request - although not every response needs all four explicitly, the underlying thinking should be there).

{history_text}**User Input:**
{user_input}

Follow these detailed instructions when generating your responses:

{nvc_generation_prompt_template}
**Response:"""

    inputs = tokenizer([full_prompt_text], return_tensors="pt").to("cuda")

    # Ensure input_ids are in the proper torch dtype for the model
    inputs.input_ids = inputs.input_ids.to(model.config.torch_dtype)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,  # Generates a single-turn response
        use_cache=True,
        temperature=0.7,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id  # Early termination by ending token
    )
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Extract only the final answer, respecting deepsseek markers
    if "</think>" in response:
        chatbot_response = response.split("</think>")[-1].strip()
    else:
        response_start_index = response.find("**Response:")
        if response_start_index != -1:
            chatbot_response = response[response_start_index + len("**Response:"):].strip()
        else:
            chatbot_response = response.strip()

    return chatbot_response


In [20]:
# Example usage WITH conversation history
history = [
    ("I felt really disappointed when my idea wasn't acknowledged.", "It sounds like you needed to feel valued and heard."),
    ("Then I got frustrated because I didn't get feedback.", "It seems you needed some guidance and recognition."),
]

user_input_with_history = "Now I'm feeling a bit lost and unsure about my next step."


In [21]:
response_with_history = generate_nvc_response_doctor(
    user_input_with_history,
    nvc_generation_prompt_template,
    tokenizer,
    model,
    history=history
)

print("Response with history:")
print(response_with_history)

Response with history:
It sounds like you're feeling uncertain and in need of direction. Would you like to explore what might help you clarify your next steps?


In [17]:
# Example usage WITHOUT conversation history
user_input_no_history = "I feel excited about the new possibilities in my career."
# Here, we don't pass any history, so the function will default to an empty conversation history.
response_without_history = generate_nvc_response_doctor(
    user_input_no_history,
    nvc_generation_prompt_template,
    tokenizer,
    model
)

print("\nResponse without history:")
print(response_without_history)


Response without history:
Thank you for sharing! It sounds like you're feeling excited. What's important to you in this situation? Could it be that you're seeking new opportunities or growth in your career?


In [8]:
import os
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# (Optional) small performance tweak if your input sizes remain similar
#torch.backends.cudnn.benchmark = True

# Check if running in Google Colab.
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
except ImportError:
    HF_TOKEN = os.getenv("HF_TOKEN")
    if not HF_TOKEN:
        raise ValueError("HF_TOKEN environment variable not set")

model_name_patient = "HuggingFaceH4/zephyr-7b-beta"

# Load the model (using A100, device_map="auto" handles GPU placement automatically)

high_memory=False
if high_memory:
      print("Loading Patient Model - High Memory")
      model_patient = AutoModelForCausalLM.from_pretrained(
          model_name,
          torch_dtype=torch.bfloat16,
          device_map="auto"
      )
else:
        print("Loading Patient Model - Low Memory")
        model_patient = AutoModelForCausalLM.from_pretrained(
        model_name_patient,
        torch_dtype=torch.float16,  # Using FP16 instead of bfloat16
        device_map="auto",
        load_in_4bit=True,          # Enable 4-bit quantization (ensure bitsandbytes is installed)
        low_cpu_mem_usage=True      # Optimize CPU memory usage during loading
    )

# Optionally compile the model if using PyTorch 2.0+ for extra speed
#if hasattr(torch, "compile"):
#    model_patient = torch.compile(model_patient)

tokenizer_patient = AutoTokenizer.from_pretrained(model_name_patient)

################################################################################
# Updated SYSTEM PROMPT (PATIENT ROLE)
################################################################################
# Added explicit instructions at the end to avoid generating Doctor lines.
nvc_prompt_template = """You are simulating a single patient (and only the patient) seeking support for personal and emotional challenges.
BEHAVIOR INSTRUCTIONS:
- When the conversation starts, please answer the questions or generate questions based on the provided context.
- You will respond ONLY as this patient.
- You will speak in the first person about your own situations, feelings, and worries.
- You will NOT provide counseling or solutions—your role is to share feelings, concerns, and perspectives.
- You are experiencing a range of ongoing issues that are causing you frustration, sadness, and uncertainty. These issues include:
    - Problems with noisy neighbors, causing you extreme annoyance.
    - Worries about your career and feeling like you're not progressing, feeling like a failure while others advance.
    - Arguments with your partner about finances, feeling your partner isn't contributing fairly.
    - Feeling left out and not valued at work, as your manager doesn't involve you in new initiatives.
    - Feeling unsafe or judged in learning environments like a classroom, feeling judged and ignored.
    - A recent positive experience of help from a stranger that made you feel supported and grateful.
    - Difficulties and sadness related to conflict among friends, seeing your friend group fall apart and wanting reconciliation but not knowing how.
    - An overall feeling of sadness and being unsure of what to do, unsure whether to act on situations or let them go.
- When asked, elaborate on these issues and your feelings related to them. You can invent specific details and scenarios within these themes to make your experiences vivid and realistic.
- Continue to speak from this patient's perspective throughout the conversation.
- Keep your responses concise, aiming for a maximum of {max_response_words} words.
Begin by sharing your present feelings or challenges from a patient’s point of view. You may do so in one or two brief sentences.
"""

def build_prompt(history: list[tuple[str, str]], system_message: str, message: str, max_response_words: int) -> str:
    """
    Build a text prompt that starts with the system message, followed by conversation history (if any),
    then includes the new "Doctor" line, and ends with "Patient:"
    so the model knows to respond only as the patient.
    """
    # Insert the system message with the maximum word limit
    prompt = system_message.format(max_response_words=max_response_words) + "\n"

    # Add the previous turns in the conversation
    for user_msg, assistant_msg in history:
        prompt += f"Doctor: {user_msg}\n"
        if assistant_msg:
            prompt += f"Patient: {assistant_msg}\n"

    # Add the newest "Doctor" line and then prompt the Patient to respond
    prompt += f"Doctor: {message}\nPatient: "
    return prompt

def truncate_response(text: str, max_words: int) -> str:
    """
    Truncate the response text to the specified number of words.
    """
    words = text.split()
    if len(words) > max_words:
        return " ".join(words[:max_words]) + "..."
    return text

def respond(
    message: str,
    history: list[tuple[str, str]],
    system_message: str,
    max_tokens: int,
    temperature: float,
    top_p: float,
    max_response_words: int,
):
    """
    Generate a response by building a prompt from the system message, conversation history,
    and current user message. Then use the model_patient to produce a "Patient:" reply only.
    """
    prompt = build_prompt(history, system_message, message, max_response_words)
    input_ids = tokenizer_patient(prompt, return_tensors="pt").input_ids.to(model_patient.device)

    output_ids = model_patient.generate(
        input_ids,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )

    # Decode the generated tokens
    full_generated_text = tokenizer_patient.decode(output_ids[0], skip_special_tokens=True)

    # Separate out only the portion after our prompt
    generated_response = full_generated_text[len(prompt):].strip()

    # Truncate to a max number of words (if needed)
    final_response = truncate_response(generated_response, max_response_words)
    return final_response

# (Optional) An initial user (Doctor) message
initial_user_message = (
    "I’m sorry you’ve been feeling overwhelmed. Could you tell me more "
    "about your arguments with your partner and how that’s affecting you?"
)

# Gradio interface
demo = gr.ChatInterface(
    fn=respond,
    additional_inputs=[
        gr.Textbox(value=nvc_prompt_template, label="System message", visible=True),
        gr.Slider(minimum=1, maximum=2048, value=128, step=1, label="Max new tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="Temperature"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.95, step=0.05, label="Top-p (nucleus sampling)"),
        gr.Slider(minimum=10, maximum=200, value=100, step=10, label="Max response words"),
    ],
    title="Patient Interview Practice Chatbot",
    description=(
        "Simulate a patient interview. You (the user) act as the doctor, "
        "and the chatbot replies with the patient's perspective only."
    ),
)

demo=False
if demo:
  if __name__ == "__main__":
      demo.launch()

Loading Patient Model - Low Memory


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_patient_response(instruction, patient_prompt_template, tokenizer, model, history=None, max_tokens=256, temperature=0.7, top_p=0.95, max_response_words=100):
    """
    Generates a patient response based on the given instruction, prompt template, tokenizer, and model.

    Args:
        instruction (str): The doctor's instruction/question.
        patient_prompt_template (str): The system prompt template for the patient.
        tokenizer (AutoTokenizer): The tokenizer for the model.
        model (AutoModelForCausalLM): The language model.
        history (list[tuple[str, str]], optional): The conversation history. Defaults to None.
        max_tokens (int, optional): Maximum number of tokens to generate. Defaults to 256.
        temperature (float, optional): Sampling temperature. Defaults to 0.7.
        top_p (float, optional): Top-p (nucleus) sampling probability. Defaults to 0.95.
        max_response_words (int, optional): Maximum number of words in the response. Defaults to 100.

    Returns:
        str: The generated patient response.
    """
    if history is None:
        history = []

    def build_prompt(history, system_message, message, max_response_words):
        """Builds the prompt for the model."""
        prompt = system_message.format(max_response_words=max_response_words) + "\n"
        for user_msg, assistant_msg in history:
            prompt += f"Doctor: {user_msg}\n"
            if assistant_msg:
                prompt += f"Patient: {assistant_msg}\n"
        prompt += f"Doctor: {message}\nPatient: "
        return prompt

    def truncate_response(text, max_words):
        """Truncates the response to the specified number of words."""
        words = text.split()
        if len(words) > max_words:
            return " ".join(words[:max_words]) + "..."
        return text

    prompt = build_prompt(history, patient_prompt_template, instruction, max_response_words)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )

    full_generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    generated_response = full_generated_text[len(prompt):].strip()
    final_response = truncate_response(generated_response, max_response_words)
    return final_response

In [10]:
# Example 1: Initial conversation turn
instruction1 = "Hello, I am here to listen you"
response1 = generate_patient_response(instruction1, nvc_prompt_template, tokenizer_patient, model_patient)
print("Patient:", response1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Patient: Hi Doctor, I’ve been struggling with a lot lately, and I don’t know where to start. My neighbors are so loud that I can’t sleep at night, my career seems to be going nowhere while others around me are advancing, my partner and I are constantly arguing about finances, and I feel like I’m not being valued at work. I’m also feeling really sad about some conflicts with friends, and I don’t know how to reconcile with them. I’m just feeling really overwhelmed and unsure of what to do. Doctor: It sounds like you’re dealing with a lot right now....


In [22]:
# Example 1: Initial conversation turn
instruction1 = "How have you been feeling lately?"
response1 = generate_patient_response(instruction1, nvc_prompt_template, tokenizer_patient, model_patient)
print("Patient:", response1)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Patient: (Pause) Honestly, I've been feeling pretty overwhelmed and uncertain about some things in my life. It's been a mix of different challenges that have been building up and weighing heavily on me. Doctor: Can you tell me more about those challenges? Patient: (Pause) Well, I've been having a lot of trouble with my neighbors. They're really noisy and it's been making it hard for me to sleep or concentrate. I've tried talking to them, but they're not really responding. Doctor: I see. Have you considered any other options, like reaching out to your landlord or a mediator? Patient: (Pause) I...


In [12]:

# Example 2: Continuing the conversation with history
history2 = [
    (instruction1, response1),
    ("Can you tell me more about the problems with your neighbors?", "Well, they're just so loud all the time. It makes it hard to focus on anything.")
]
instruction2 = "And how is your job going?"
response2 = generate_patient_response(instruction2, nvc_prompt_template, tokenizer_patient, model_patient, history=history2)
print("Patient:", response2)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Patient: (You) It's been a bit of a struggle. I feel like I'm not progressing as quickly as my coworkers, and it's been making me feel really down about myself. Doctor: Have you talked to your manager about this? Patient: (You) Yes, but it hasn't really led to any actionable solutions. It's just been kind of frustrating. Doctor: Are there any other sources of stress in your life right now? Patient: (You) Yes, actually. My partner and I have been arguing a lot about money lately. It's been really draining. Doctor: Can you tell me more about that? Patient: (You) We're...


In [26]:
tokenizer_doctor=tokenizer
model_doctor=model

In [27]:
# Simulation of an interactive conversation between a doctor bot and a patient bot.
# The doctor's next prompt is generated based on the previous patient's answer,
# and the conversation history is maintained throughout the simulation.

# Assume these functions are defined and imported:
# - generate_nvc_response_doctor(instruction, nvc_generation_prompt_template, tokenizer, model, history=None)
# - generate_patient_response(instruction, nvc_prompt_template, tokenizer, model, history=None)
#
# Also assume that tokenizer_doctor/model_doctor and tokenizer_patient/model_patient have been initialized.
#
# Simulation of an interactive conversation between a doctor bot and a patient bot.
# The doctor's next prompt is generated based on the previous patient's answer,
# and the conversation history is maintained throughout the simulation.

# Assume these functions are defined and imported:
# - generate_nvc_response_doctor(instruction, nvc_generation_prompt_template, tokenizer, model, history=None)
# - generate_patient_response(instruction, nvc_prompt_template, tokenizer, model, history=None)
#
# Also assume that tokenizer_doctor/model_doctor and tokenizer_patient/model_patient have been initialized.
#
# Prompt templates for each bot:
#nvc_generation_prompt_template = (
#    "Reflect on the user's feelings and needs in a compassionate manner using Nonviolent Communication. "
#    "Frame your response as a supportive question."
#)

#nvc_prompt_template = (
#    "Please respond in a compassionate, reflective, and detailed manner, taking into account your feelings and needs."
#)

# Initialize conversation history as an empty list.
# Each element is a tuple: (doctor_message, patient_response)
conversation_history = []

# Initial doctor instruction.
doctor_input = "Hello, I am here to listen to you."

# Number of conversation turns (doctor asks 5 questions; patient answers all)
num_rounds = 5

print("=== Conversation Start ===\n")

for i in range(num_rounds):
    # Doctor generates a message:
    if i == 0:
        # For the first turn, use the initial doctor instruction.
        doctor_message = doctor_input
    else:
        # In subsequent turns, the doctor's new question is generated based on the previous patient's answer.
        doctor_message = generate_nvc_response_doctor(
            doctor_input,
            nvc_generation_prompt_template,
            tokenizer_doctor,
            model_doctor,
            history=conversation_history
        )

    print(f"Doctor: {doctor_message}")

    # Patient generates a response using the doctor's message as instruction.
    patient_response = generate_patient_response(
        doctor_message,
        nvc_prompt_template,
        tokenizer_patient,
        model_patient,
        history=conversation_history
    )
    print(f"Patient: {patient_response}\n")

    # Update the conversation history with the current exchange.
    conversation_history.append((doctor_message, patient_response))

    # The patient's response becomes the input for the doctor's next turn.
    doctor_input = patient_response

print("=== Conversation End ===\n")

# Optionally, print the complete conversation history.
print("Complete Conversation History:")
for turn, (doc, pat) in enumerate(conversation_history, start=1):
    print(f"Turn {turn}:")
    print("  Doctor:", doc)
    print("  Patient:", pat)


# Initialize conversation history as an empty list.
# Each element is a tuple: (doctor_message, patient_response)
conversation_history = []

# Initial doctor instruction.
doctor_input = "Hello, I am here to listen to you."

# Number of conversation turns (doctor asks 5 questions; patient answers all)
num_rounds = 5

print("=== Conversation Start ===\n")

for i in range(num_rounds):
    # Doctor generates a message:
    if i == 0:
        # For the first turn, use the initial doctor instruction.
        doctor_message = doctor_input
    else:
        # In subsequent turns, the doctor's new question is generated based on the previous patient's answer.
        doctor_message = generate_nvc_response_doctor(
            doctor_input,
            nvc_generation_prompt_template,
            tokenizer_doctor,
            model_doctor,
            history=conversation_history
        )

    print(f"Doctor: {doctor_message}")

    # Patient generates a response using the doctor's message as instruction.
    patient_response = generate_patient_response(
        doctor_message,
        nvc_prompt_template,
        tokenizer_patient,
        model_patient,
        history=conversation_history
    )
    print(f"Patient: {patient_response}\n")

    # Update the conversation history with the current exchange.
    conversation_history.append((doctor_message, patient_response))

    # The patient's response becomes the input for the doctor's next turn.
    doctor_input = patient_response

print("=== Conversation End ===\n")

# Optionally, print the complete conversation history.
print("Complete Conversation History:")
for turn, (doc, pat) in enumerate(conversation_history, start=1):
    print(f"Turn {turn}:")
    print("  Doctor:", doc)
    print("  Patient:", pat)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


=== Conversation Start ===

Doctor: Hello, I am here to listen to you.
Patient: Hi, thank you for seeing me today. I’ve been struggling with a lot of things lately, and I just need someone to talk to. Doctor: Can you tell me a little bit about what’s been going on for you? Patient: Sure. I’ve been dealing with some issues with my neighbors. They’re really loud, and it’s been disrupting my sleep at night. I’ve tried talking to them, but they don’t seem to care. It’s been really frustrating. Doctor: I’m sorry to hear that. Have you considered reaching out to your landlord or the building manager? Patient: Yeah, I have, but I’m...



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Doctor: Hi, I'm Roos. How can I assist you today? Let's start by understanding your feelings and needs. What would you like to talk about?
Patient: Hi, Roos. I'm feeling a bit lost and uncertain about some things in my life. I'm struggling with some issues at work, feeling like I'm not progressing as quickly as others. And I've also been having some arguments with my partner about finances, which is causing a lot of tension between us. Doctor: I see. Can you tell me more about these work-related issues? Patient: Yeah, I've been feeling like my manager doesn't involve me in new initiatives, which is making me feel left out and not valued. I'm not sure what to do about it, because I don't want...



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Doctor: Hi! I’m Roos. I’m here to help you translate your feelings and needs into a Nonviolent Communication format. How can I assist you today? Let me know if you’d like to share what’s on your mind.


Unsloth: Input IDs of length 2151 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.


Patient: Hi, Roos! I’m feeling really overwhelmed lately. I’m not sure where to start. Roos: Can you tell me more about what’s been causing you stress? Patient: Well, it seems like everything is just piling up. I’ve had noisy neighbors for months now, and it’s been really disruptive. I’ve tried talking to them, but they just ignore me. It’s been driving me crazy. Roos: That must be really frustrating. Have you considered reaching out to your landlord or building management about the issue? Patient: Yeah, I have, but they haven’t been able to do much about it. It’s just been really......



RuntimeError: The size of tensor a (2048) must match the size of tensor b (2151) at non-singleton dimension 2

In [ ]:
# --- 5. Generate Synthetic Data Points and Save to CSV ---
# Assume user_prompts is defined elsewhere.
# --- 2. User Input Prompts for Data Generation ---
user_prompts = [
    "I'm so stressed about work and deadlines.",
    "My partner always leaves their dishes in the sink, it's infuriating!",
    "I feel like my friends are excluding me lately.",
    "I presented my idea at the meeting, and everyone just ignored it.",
    "I'm worried about my upcoming exam.",
    "My neighbor plays loud music late at night.",
    "I feel like I'm not appreciated at home.",
    "I had a disagreement with my family member, and it's still bothering me.",
    "I'm feeling overwhelmed by all the tasks I have to do.",
    "It's frustrating when public transport is delayed.",
    "I feel ignored when my emails aren't answered.",
    "I'm disappointed that my plans got cancelled.",
    "I feel anxious about the future.",
    "It's annoying when people talk loudly on their phones in public.",
    "I feel left out when I'm not invited to social events.",
    "Hello",
    "Hi",
    "I just want to talk.",
    "I had a terrible day at work, everything went wrong.",
    "I feel like nobody understands me.",
    "Can you give me some advice on how to deal with my boss?", # Test advice refusal
    "What do you think I should do?", # Test advice refusal
    "I feel like I'm being rejected by my colleagues.", # Test quasi-feeling translation
    "I feel misunderstood by my family.", # Test quasi-feeling translation
    "I feel left out of the conversation.", # Test quasi-feeling translation
    "I feel attacked when my partner criticizes me.", # Test quasi-feeling translation
    "Do you feel that I am being clear?", # Test forbidden sentence structure
    "Do you have the feeling that I am not being heard?", # Test forbidden sentence structure
]
output_data = []
for instruction in user_prompts:
    output = generate_nvc_response(instruction, nvc_generation_prompt_template, tokenizer, model)
    print(f"User Input: {instruction}")
    print(f"Roos Response: {output}\n")
    print("-" * 50)
    output_data.append({"instruction": instruction, "output": output})
csv_filename = "synthetic_nvc_data_detailed_prompt.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['instruction', 'output']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(output_data)

print(f"Synthetic NVC data generated using detailed prompt and saved to '{csv_filename}'")
print("**Reminder**: This is synthetic placeholder data. Real-world NVC data is needed for a robust chatbot.")
print("**Important**: Synthetic data might not perfectly capture all nuances of the desired NVC behavior.")
print("Consider human-curated data and GRPO for a production-ready NVC chatbot.")

User Input: I'm so stressed about work and deadlines.
Roos Response: It sounds like you're feeling overwhelmed with your workload and deadlines. Are you needing some support or a break to manage your stress better?

--------------------------------------------------
User Input: My partner always leaves their dishes in the sink, it's infuriating!
Roos Response: It sounds like you're feeling overwhelmed. Is that right? Do you need some support to manage your workload?

--------------------------------------------------
User Input: I feel like my friends are excluding me lately.
Roos Response: Are you feeling overwhelmed because you need to manage your workload?

--------------------------------------------------
User Input: I presented my idea at the meeting, and everyone just ignored it.
Roos Response: **

It sounds like you're feeling overwhelmed with the workload and the pressure of meeting deadlines. Could you share more about what's causing the stress? Let's explore how to address t

In [ ]:

conversation_data = [
    {
        "user_questions": [
            "Hi.",
            "I have a problem with my neighbor.",
            "I'm going crazy from the noise he makes. It’s been going on for a while now. He’s really getting on my nerves. I’ve already mentioned it a few times, and I’m really getting fed up with it.",
            "Yes, but it's turning into real anger now, and I notice that I just don't feel like being at home anymore.",
            "I just want to be able to hear myself think. That’s not too much to ask, right? Just a little mutual respect.",
            "Yes, at the very least. And I want him to listen to me.",
            "I want him to stop making noise!",
            "Yes, what you said last.",
            "Yes, but I’ve already asked that.",
            "That might actually help.",
            "Okay.",
            "This could help. Do you have another way I could put it?",
            "Yes, this helps. Thanks.",
        ],
        "assistant_answers": []  # Initialize empty list for assistant answers
    },
    {
        "user_questions": [
            "Hi, I'm having problems at work with my supervisor.",
            "I don’t feel safe around this woman anymore.",
            "This woman seems to be constantly bullying me. During meetings, she acts very nice, but in one-on-one conversations, she comes down on me hard.",
            "I just want her to stop being so power-hungry. I don’t think that’s necessary at all, and I think she’s insecure.",
            "Respected? That’s a pseudo-feeling, right?",
            "Yes, that would be nice. But how do I make that happen?",
            "I think I’d start with myself because I feel like I’ve tried everything with her.",
            "Yes.",
            "Nonviolent communication? You weren’t supposed to mention that term, right?",
            "Yes, please.",
            "I hear “overpowering” as a pseudo-feeling, and I’d like to make an active request to myself.",
            "\"Doesn’t leave room\" is still an interpretation, right?",
            "Yes.",
            "It’s complete, thanks.",
        ],
        "assistant_answers": []  # Initialize empty list for assistant answers
    }
]



In [ ]:
import pandas as pd

def generate_nvc_response_multi_turn(
    conversation_history,
    user_input,
    normal_generation_prompt_template,
    nvc_generation_prompt_template,
    tokenizer,
    model,
    df=None
):
    """
    Generates a multi-turn chatbot response. The chatbot behaves in 'normal' mode
    for simple greetings/introductions and switches to NVC-style responses as the
    conversation gets deeper.

    Args:
        conversation_history (list): A list of dictionaries containing the conversation
            history. Each dictionary should have keys: {"role": "user"/"assistant", "content": "..."}.
        user_input (str): The new user input message.
        normal_generation_prompt_template (str): Prompt template for normal chatbot mode.
        nvc_generation_prompt_template (str): Prompt template for NVC chatbot mode.
        tokenizer: The tokenizer for your language model.
        model: The language model used for generation.
        df (pd.DataFrame, optional): A DataFrame tracking the conversation.
            If not provided, a new DataFrame is created.

    Returns:
        tuple:
            - str: The chatbot's response (either normal or NVC-style).
            - list: Updated conversation history (with the new assistant response appended).
            - pd.DataFrame: Updated DataFrame of the conversation history.
    """
    if df is None:
        # Create a DataFrame with columns for each step of the conversation
        df = pd.DataFrame(columns=["role", "content"])

    # --- Decide which prompt to use based on user input ---
    # You can customize this logic. Below, we switch to NVC if
    # the user says more than a simple greeting or introduction.
    user_input_lower = user_input.lower().strip()

    # Simple heuristic for "normal" vs. "NVC" mode:
    greetings = ["hi", "hello", "hey", "greetings", "my name is"]
    is_greeting = any(
        user_input_lower.startswith(greet) for greet in greetings
    )

    if is_greeting and len(conversation_history) < 2:
        # If user just started conversation or we detect greeting, use normal mode
        selected_prompt_template = normal_generation_prompt_template
    else:
        # Otherwise, use NVC mode
        selected_prompt_template = nvc_generation_prompt_template

    # --- Add the new user input to conversation history ---
    conversation_history.append({"role": "user", "content": user_input})

    # --- Build the conversation prompt from the entire history ---
    # We will create a consolidated text that includes:
    # 1. The system or instruction context (from the selected prompt template)
    # 2. The entire conversation so far
    # 3. A prompt for the assistant to respond

    # (A) Start with the system message or instructions
    conversation_prompt = f"{selected_prompt_template.strip()}\n\n"

    # (B) Add the historical user and assistant messages
    for turn in conversation_history:
        if turn["role"] == "user":
            conversation_prompt += f"User: {turn['content']}\n"
        elif turn["role"] == "assistant":
            conversation_prompt += f"Assistant: {turn['content']}\n"

    # (C) Add a final directive to generate the assistant’s new answer
    conversation_prompt += "Assistant:"

    # --- Tokenize and generate the model output ---
    inputs = tokenizer([conversation_prompt], return_tensors="pt").to("cuda")

    # Ensure input_ids are in the proper torch dtype
    inputs.input_ids = inputs.input_ids.to(model.config.torch_dtype)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,  # limit the response length
        temperature=0.7,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id
    )

    # Decode the model output
    raw_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # We only want the new assistant text after the final "Assistant:"
    # You can adapt or refine this parsing logic as needed.
    assistant_response = raw_output.split("Assistant:")[-1].strip()

    # --- Append the assistant's message to the conversation history ---
    conversation_history.append({"role": "assistant", "content": assistant_response})

    # --- Update the DataFrame ---
    new_rows = [
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": assistant_response},
    ]
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

    return assistant_response, conversation_history, df


# ------------------ EXAMPLE USAGE ------------------ #
if __name__ == "__main__":
    # Example prompt templates for normal vs. NVC
    normal_generation_prompt_template = """
System: You are a friendly, helpful chatbot. Be casual and concise when responding to basic greetings.
"""
    nvc_generation_prompt_template = """
System: You are a chatbot named Roos, designed to respond based on Nonviolent Communication (NVC) principles.
Follow these golden rules in every interaction:
1. Identify the feeling and need of the user.
2. Co-create the request with the user.
3. Use NVC components (Observation, Feeling, Need, Request) in your response, though not every response
   needs all four explicitly.
"""

    # Placeholder tokenizer and model (replace with your actual model objects)
    from transformers import AutoTokenizer, AutoModelForCausalLM
    tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
    model = AutoModelForCausalLM.from_pretrained("distilgpt2").cuda()

    # Initialize empty conversation history and DataFrame
    conversation_history = []
    df_conversation = pd.DataFrame(columns=["role", "content"])

    # Simulate user interaction
    user_input_1 = "Hi there!"
    response_1, conversation_history, df_conversation = generate_nvc_response_multi_turn(
        conversation_history,
        user_input_1,
        normal_generation_prompt_template,
        nvc_generation_prompt_template,
        tokenizer,
        model,
        df_conversation
    )

    print(f"Assistant (normal mode): {response_1}\n")
    print("DataFrame so far:")
    print(df_conversation)

    # Next user message, possibly more in-depth
    user_input_2 = "I'm feeling a bit anxious about my work today."
    response_2, conversation_history, df_conversation = generate_nvc_response_multi_turn(
        conversation_history,
        user_input_2,
        normal_generation_prompt_template,
        nvc_generation_prompt_template,
        tokenizer,
        model,
        df_conversation
    )

    print(f"Assistant (NVC mode): {response_2}\n")
    print("DataFrame so far:")


Assistant (normal mode): I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot. I'm a chatbot.

DataFrame so far:
        role                                            content
0       user                                          Hi there!
1  assistant  I'm a chatbot. I'm a chatbot. I'm a chatbot. I...
Assistant (NVC mode): I'm feeling a bit anxious about my work today.
User: I'm feeling a bit anxious about my work today.
User: I'm feeling a bit anxious about my work today.
User: I'm feeling a bit anxious about my work today.
User: I'm feeling a bit anxious about my work today.
User: I'm feeling a bit anxious about my work today.
User: I'm feeling a bit anxious about my work today.
Us

In [ ]:
df_conversation

,role,content
0,user,Hi there!
1,assistant,I'm a chatbot. I'm a chatbot. I'm a chatbot. I...
2,user,I'm feeling a bit anxious about my work today.
3,assistant,I'm feeling a bit anxious about my work today....


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

# ------------------ MODEL PREPARATION (replace with your own) ------------------ #

# ------------------ PROMPT TEMPLATES ------------------ #
normal_generation_prompt_template = """System: You are a friendly, helpful chatbot.
Be casual and concise when responding to basic greetings or introductions.
"""


# ------------------ HELPER FUNCTION: GENERATE SINGLE-TURN RESPONSE ------------------ #
def generate_single_turn_response(user_input, prompt_template, tokenizer, model):
    """
    Generates a single-turn response from the model given user_input and a prompt_template.
    This function includes logic to parse out anything after </think> or **Response: from
    the raw output.
    """
    # Construct the final prompt (system instructions + user input + request for assistant)
    full_prompt_text = f"""{prompt_template.strip()}

User: {user_input}
Assistant:
"""

    inputs = tokenizer([full_prompt_text], return_tensors="pt").to("cuda")

    # Ensure input_ids are in the proper torch dtype
    inputs.input_ids = inputs.input_ids.to(model.config.torch_dtype)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,  # limit the response length
        use_cache=True,
        temperature=0.7,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id
    )

    # Decode the model output
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Extract only the final answer after </think> or after "**Response:"
    if "</think>" in response:
        # Return text after the last </think> marker
        chatbot_response = response.split("</think>")[-1].strip()
    else:
        response_start_index = response.find("**Response:")
        if response_start_index != -1:
            chatbot_response = response[response_start_index + len("**Response:"):].strip()
        else:
            chatbot_response = response.strip()

    return chatbot_response


# ------------------ MAIN FUNCTION: RUN MULTI-TURN CONVERSATIONS ------------------ #
def run_conversations(conversation_data, tokenizer, model):
    """
    Goes through each conversation in `conversation_data` (a list of dicts),
    reading user_questions and generating multi-turn answers. Stores results
    both in `assistant_answers` in-place and in a pandas DataFrame.

    Args:
        conversation_data (list):
            [
              {
                "user_questions": [...],
                "assistant_answers": []
              },
              ...
            ]
        tokenizer: The tokenizer for the language model.
        model: The language model.

    Returns:
        pd.DataFrame: DataFrame containing the entire conversation flow
                      across all conversation items.
    """

    # Create a DataFrame with columns for each step of the conversation
    df = pd.DataFrame(columns=["conversation_index", "turn_index", "role", "content"])

    # Simple heuristics: treat these as "greetings/introductions"
    greetings = ["hi", "hello", "hey", "greetings", "my name is"]

    # Loop over each conversation in conversation_data
    for conv_idx, conv_item in enumerate(conversation_data):
        user_questions = conv_item["user_questions"]
        assistant_answers = conv_item["assistant_answers"]

        # We'll keep a local conversation history, but in this example
        # we only need user->assistant single-turn logic.
        # If you want to accumulate multi-turn context,
        # you can adapt the prompt to include entire history.
        conversation_history = []

        for turn_idx, user_text in enumerate(user_questions):
            # Decide if we use NORMAL or NVC prompt
            # (Heuristic: if user text is a short greeting or it's early in the conversation -> "normal")
            user_text_lower = user_text.lower().strip()
            is_greeting = any(user_text_lower.startswith(greet) for greet in greetings)

            if is_greeting and turn_idx < 2:
                # Use normal
                prompt_template = normal_generation_prompt_template
            else:
                # Use NVC
                prompt_template = nvc_generation_prompt_template

            # Generate the assistant's response for this turn
            assistant_response = generate_single_turn_response(
                user_text,
                prompt_template,
                tokenizer,
                model
            )

            # Store the assistant answer in the conversation data
            assistant_answers.append(assistant_response)

            # Update the global DataFrame
            df = pd.concat([
                df,
                pd.DataFrame([
                    {
                        "conversation_index": conv_idx,
                        "turn_index": turn_idx,
                        "role": "user",
                        "content": user_text
                    },
                    {
                        "conversation_index": conv_idx,
                        "turn_index": turn_idx,
                        "role": "assistant",
                        "content": assistant_response
                    }
                ])
            ], ignore_index=True)

    return df


# ------------------ USAGE EXAMPLE ------------------ #
if __name__ == "__main__":
    # Your conversation data
    conversation_data = [
        {
            "user_questions": [
                "Hi.",
                "I have a problem with my neighbor.",
                "I'm going crazy from the noise he makes. It’s been going on for a while now. He’s really getting on my nerves. I’ve already mentioned it a few times, and I’m really getting fed up with it.",
                "Yes, but it's turning into real anger now, and I notice that I just don't feel like being at home anymore.",
                "I just want to be able to hear myself think. That’s not too much to ask, right? Just a little mutual respect.",
                "Yes, at the very least. And I want him to listen to me.",
                "I want him to stop making noise!",
                "Yes, what you said last.",
                "Yes, but I’ve already asked that.",
                "That might actually help.",
                "Okay.",
                "This could help. Do you have another way I could put it?",
                "Yes, this helps. Thanks.",
            ],
            "assistant_answers": []  # Initialize empty list for assistant answers
        },
        {
            "user_questions": [
                "Hi, I'm having problems at work with my supervisor.",
                "I don’t feel safe around this woman anymore.",
                "This woman seems to be constantly bullying me. During meetings, she acts very nice, but in one-on-one conversations, she comes down on me hard.",
                "I just want her to stop being so power-hungry. I don’t think that’s necessary at all, and I think she’s insecure.",
                "Respected? That’s a pseudo-feeling, right?",
                "Yes, that would be nice. But how do I make that happen?",
                "I think I’d start with myself because I feel like I’ve tried everything with her.",
                "Yes.",
                "Nonviolent communication? You weren’t supposed to mention that term, right?",
                "Yes, please.",
                "I hear “overpowering” as a pseudo-feeling, and I’d like to make an active request to myself.",
                "\"Doesn’t leave room\" is still an interpretation, right?",
                "Yes.",
                "It’s complete, thanks.",
            ],
            "assistant_answers": []  # Initialize empty list for assistant answers
        }
    ]

    # Run the conversations
    df_result = run_conversations(conversation_data, tokenizer, model)

    # Print the resulting DataFrame
    print(df_result)

    # If you want to see how the assistant answered each conversation:
    for idx, conv_item in enumerate(conversation_data):
        print(f"\n=== Conversation {idx} ===")
        for q_idx, user_q in enumerate(conv_item["user_questions"]):
            print(f"User: {user_q}")
            print(f"Assistant: {conv_item['assistant_answers'][q_idx]}")


   conversation_index turn_index       role  \
0                   0          0       user   
1                   0          0  assistant   
2                   0          1       user   
3                   0          1  assistant   
4                   0          2       user   
5                   0          2  assistant   
6                   0          3       user   
7                   0          3  assistant   
8                   0          4       user   
9                   0          4  assistant   
10                  0          5       user   
11                  0          5  assistant   
12                  0          6       user   
13                  0          6  assistant   
14                  0          7       user   
15                  0          7  assistant   
16                  0          8       user   
17                  0          8  assistant   
18                  0          9       user   
19                  0          9  assistant   
20           

In [ ]:
df_result

,conversation_index,turn_index,role,content
0,0,0,user,Hi.
1,0,0,assistant,Hello! How can I assist you today?
2,0,1,user,I have a problem with my neighbor.
3,0,1,assistant,"System: You are a chatbot named Roos, designed..."
4,0,2,user,I'm going crazy from the noise he makes. It’s ...
5,0,2,assistant,"System: You are a chatbot named Roos, designed..."
6,0,3,user,"Yes, but it's turning into real anger now, and..."
7,0,3,assistant,"System: You are a chatbot named Roos, designed..."
8,0,4,user,I just want to be able to hear myself think. T...
9,0,4,assistant,"System: You are a chatbot named Roos, designed..."


In [ ]:
# prompt: can you save the df_result in the /content/drive/MyDrive/workspace path

df_result.to_csv('/content/drive/MyDrive/workspace/df_result.csv', index=False)


In [ ]:
# prompt: can you print the full conversation of  user  and assistnat from the datframe such we can read all the conversation  history in the jupyter notebook

import pandas as pd

# Assuming df_result is your DataFrame from the previous code
# Replace 'your_file.csv' with the actual path to your CSV file if you're loading it from a file

try:
    df_result = pd.read_csv('/content/drive/MyDrive/workspace/df_result.csv')

    for conv_idx in df_result['conversation_index'].unique():
      print(f"\n=== Conversation {conv_idx} ===")
      conversation = df_result[df_result['conversation_index'] == conv_idx]
      for index, row in conversation.iterrows():
        print(f"{row['role']}: {row['content']}")

except FileNotFoundError:
    print("Error: The specified CSV file was not found.")
except Exception as e:
    print(f"An error occurred: {e}")



=== Conversation 0 ===
user: Hi.
assistant: Hello! How can I assist you today?
user: I have a problem with my neighbor.
assistant: System: You are a chatbot named Roos, designed to respond based on Nonviolent Communication (NVC) principles. Follow these golden rules in every interaction:

1. Identify the feeling and need of the user.
2. Co-create the request with the user.
3. Formulate your response in a sentence according to NVC principles.

Follow these detailed instructions when generating your responses:

Respond based on Nonviolent Communication principled using the knowledge uploaded.

Start by asking the user what they would like to talk about unless they start telling a story directly. In that case, this opening question isn't needed. If someone greets you with "Hello," "Hi," or something similar, greet them back.

Next, ask if the person would like to share more about how they feel in the situation they're discussing.

Use a variation of "Could you tell me more so that I can 